In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import gc
import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

In [ ]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv("../input/train_v2.csv")

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split( ) for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
label_map

inv_label_map = {i: l for l, i in label_map.items()}

In [ ]:
for f , tags in tqdm(df_train.values, miniters=1000):
    print (f, tags)

In [ ]:
for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('../input/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(cv2.resize(img,(32,32)))
    y_train.append(targets)

In [ ]:
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

In [ ]:
print (x_train.shape)
print (y_train.shape)

In [ ]:
split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split],x_train[split:] ,y_train[:split],y_train[split:]

In [ ]:
print (x_train.shape)
print (x_valid.shape)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=(32,32,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=4, verbose=1, validation_data=(x_valid,y_valid))

In [ ]:
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)
print(y_valid)
print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))